In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
print(sys.executable)


In [ ]:
pre = "MH100"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

In [ ]:
import sccoda
from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz
import sccoda.datasets as scd

### Human

In [ ]:
def make_df_annotation_azimut1_5_scanvi_v2(df):
    df['cluster_azimut1_5_scanvi_v2'] = df['cluster_azimut1_5_scanvi'].replace(
    {'NK': 'NK+Proliferating', 'NK Proliferating': 'NK+Proliferating'})
    return df

In [ ]:
path_clusteranno_s0140 = 'F:\\monkey_IZI\\analysis\\tables\\s0140_clusteranno_scvi.txt.gz'
clusteranno_s0140 = pd.read_csv(path_clusteranno_s0140,compression = 'gzip',index_col=0)
adata_human.obs = pd.merge(adata_human.obs, clusteranno_s0140, left_index=True, right_index=True)

In [ ]:
species = 'human'
_,_,_,base_anndata_objects_H24 = h.return_local_paths(drive,pre='H24',add_path = False)
adata_human = sc.read_h5ad(os.path.join(base_anndata_objects_H24,'H24' + '_' + species + '_anno_celltypes_v0.h5ad'))

path_clusteranno_s0140 = 'F:\\monkey_IZI\\analysis\\tables\\s0140_clusteranno_scvi.txt.gz'
clusteranno_s0140 = pd.read_csv(path_clusteranno_s0140,compression = 'gzip',index_col=0)
adata_human.obs = pd.merge(adata_human.obs, clusteranno_s0140, left_index=True, right_index=True)
adata_human.obs = make_df_annotation_azimut1_5_scanvi_v2(adata_human.obs)
adata_human.obs.rename(columns={"timepoint_x": "timepoint", "individual_x": "individual","species_x" : "species"}, inplace=True)
ct_column = 'cluster_azimut1_5_scanvi_v2'
df_human =  adata_human.obs[[ct_column,'timepoint','individual']]
cell_counts_human = df_human.groupby([ct_column,'timepoint']).size().reset_index(name='count')

cell_counts_pivot_human = cell_counts_human.pivot(index='timepoint',columns = ct_column,values = 'count').reset_index()

In [ ]:
scoda_obj_human = dat.from_scanpy(
    adata_human,
    cell_type_identifier=ct_column,
    sample_identifier="timepoint",
    )

#viz.stacked_barplot(data_scanpy_1 , feature_name="samples")

In [ ]:
save_path_human_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_human.h5ad')
scoda_obj_human.write_h5ad(save_path_human_h5ad)

In [ ]:
df_all_human = pd.DataFrame(scoda_obj_human.X,columns = scoda_obj_human.var.index,index = scoda_obj_human.obs.index)
proportions = df_all_human.div(df_all_human.sum(axis=1), axis=0)
proportions = proportions.T

proportions.to_csv(os.path.join(base_table_path,pre + '_proportions_human.csv'))

In [ ]:
color_timepoints = ["#3b7c70", "#ce9642",  "#3b3a3e"] 
proportions.plot(kind='bar', figsize=(8, 6), width=0.8, color=color_timepoints)
plt.title('Cell type proportions - Human')
plt.xlabel('Cell type')
plt.ylabel('Proportion')
plt.xticks(rotation=90)
plt.ylim([0,0.7])
plt.savefig(os.path.join(base_plots_path,pre + '_composition_barplot_human.pdf'),bbox_inches = 'tight')

In [ ]:
plt.figure()
viz.stacked_barplot(scoda_obj_human , feature_name="samples")
plt.title('human')
plt.savefig(os.path.join(base_plots_path,pre + '_scoda_composition_human.pdf'),bbox_inches = 'tight')
plt.show()

In [ ]:
scoda_obj_human.obs['Condition'] = scoda_obj_human.obs.index 
scoda_model = mod.CompositionalAnalysis(scoda_obj_human, formula="Condition", reference_cell_type="automatic")

sim_results = scoda_model.sample_hmc()

sim_results.summary() 

sim_results.set_fdr(est_fdr=0.25)
print(sim_results.summary())

print(sim_results.credible_effects())

# saving
save_path = os.path.join(base_table_path,pre + '_scoda_sim_results_human.pkl')
sim_results.save(save_path)

'''
# loading
with open(path, "rb") as f:
    sim_results_loaded = pkl.load(f)
'''    

### Cyno

In [ ]:
species = 'cyno'
_,_,_,base_anndata_objects_M24 = h.return_local_paths(drive,pre='M24',add_path = False)
adata_cyno = sc.read_h5ad(os.path.join(base_anndata_objects_M24,'M24' + '_' + species + '_anno_celltypes_v0.h5ad'))

In [ ]:
path_clusteranno_s0140 = 'F:\\monkey_IZI\\analysis\\tables\\s0140_clusteranno_scvi.txt.gz'
clusteranno_s0140 = pd.read_csv(path_clusteranno_s0140,compression = 'gzip',index_col=0)
adata_cyno.obs = pd.merge(adata_cyno.obs, clusteranno_s0140, left_index=True, right_index=True)
adata_cyno.obs = make_df_annotation_azimut1_5_scanvi_v2(adata_cyno.obs)
adata_cyno.obs.rename(columns={"timepoint_x": "timepoint", "individual_x": "individual","species_x" : "species"}, inplace=True)

In [ ]:
ct_column = 'cluster_azimut1_5_scanvi_v2'

In [ ]:
df_cyno =  adata_cyno.obs[[ct_column,'timepoint','individual']]
cell_counts_cyno = df_cyno.groupby([ct_column,'timepoint']).size().reset_index(name='count')

cell_counts_pivot_cyno = cell_counts_cyno.pivot(index='timepoint',columns = ct_column,values = 'count').reset_index()

In [ ]:
scoda_obj_cyno = dat.from_scanpy(
    adata_cyno,
    cell_type_identifier=ct_column,
    sample_identifier="timepoint",
    )


In [ ]:
save_path_cyno_h5ad = os.path.join(base_anndata_objects,pre + '_scoda_obj_cyno.h5ad')
scoda_obj_cyno.write_h5ad(save_path_cyno_h5ad)

In [ ]:
df_all_cyno = pd.DataFrame(scoda_obj_cyno.X,columns = scoda_obj_cyno.var.index,index = scoda_obj_cyno.obs.index)
proportions = df_all_cyno.div(df_all_cyno.sum(axis=1), axis=0)
proportions = proportions.T

proportions.to_csv(os.path.join(base_table_path,pre + '_proportions_cyno.csv'))

In [ ]:
color_timepoints = ["#3b7c70", "#ce9642",  "#3b3a3e"] 
proportions.plot(kind='bar', figsize=(8, 6), width=0.8, color=color_timepoints)
plt.title('Cell type proportions - cyno')
plt.xlabel('Cell type')
plt.ylabel('Proportion')
plt.xticks(rotation=90)
plt.ylim([0,0.7])
plt.savefig(os.path.join(base_plots_path,pre + '_composition_barplot_cyno.pdf'),bbox_inches = 'tight')

In [ ]:
plt.figure()
viz.stacked_barplot(scoda_obj_cyno , feature_name="samples")
plt.title('cyno')
plt.savefig(os.path.join(base_plots_path,pre + '_scoda_composition_cyno.pdf'),bbox_inches = 'tight')
plt.show()

### Fitting sccoda model

In [ ]:
scoda_obj_cyno.obs['Condition'] = scoda_obj_cyno.obs.index 
scoda_model = mod.CompositionalAnalysis(scoda_obj_cyno, formula="Condition", reference_cell_type="automatic")

sim_results = scoda_model.sample_hmc()

sim_results.summary() 

sim_results.set_fdr(est_fdr=0.25)
print(sim_results.summary())

print(sim_results.credible_effects())

# saving
save_path = os.path.join(base_table_path,pre + '_scoda_sim_results_cyno.pkl')
sim_results.save(save_path)

'''
# loading
with open(path, "rb") as f:
    sim_results_loaded = pkl.load(f)
'''    

In [ ]:
# Save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()